## Test File for _project_and_slide_moving functionality

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from planktos import swarm
from planktos.geom import seg_intersect_2D_multilinear_poly as seg_intersect

# Create a function that will plot the geometry
def plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit=None, newstartpt=None, newendpt=None):
    # unpack intersection
    x = intersection[0]    # (x,y) coordinates of intersection
    t_I = intersection[1]  # btwn 0 & 1, fraction of movement traveled so far
    Q0 = intersection[2]   # edge of mesh element at time of intersection
    Q1 = intersection[3]   # edge of mesh element at time of intersection
    idx = intersection[4]  # index into close_mesh_start/end above
    
    # plot start and end position of mesh
    plt.plot(start_mesh[:,:,0].flatten(), start_mesh[:,:,1].flatten(), 'o-', color='lightgrey', label='startmesh')
    plt.plot(end_mesh[:,:,0].flatten(), end_mesh[:,:,1].flatten(), 'o-', color='black', label='endmesh')
    # plot original start and end position of agent
    plt.plot((startpt[0], endpt[0]), (startpt[1], endpt[1]), 'o-', label='orig trajectory')
    # plot element that was intersected at time of intersection
    plt.plot((Q0[0], Q1[0]), (Q0[1], Q1[1]), label='element at intersection')

    # if a critical time is given, plot the relevant mesh element at that time as well
    if t_crit is not None:
        interp_mesh_el = start_mesh[idx]*(1-t_crit) + end_mesh[idx]*t_crit
        plt.plot((interp_mesh_el[0,0],interp_mesh_el[1,0]),(interp_mesh_el[0,1],interp_mesh_el[1,1]), label='element at t_crit')
    # if new start and end points are given, plot the new trajectory off the element
    if newstartpt is not None:
        plt.plot((newstartpt[0], newendpt[0]), (newstartpt[1], newendpt[1]), 'o-', color='goldenrod', label='new trajectory')
    
    # plot point of intersection
    plt.plot(x[0], x[1], 'x', color='black')
    # plot final point
    plt.plot(finalendpt[0], finalendpt[1], 's', color='black')
    

In [ ]:
# in-line mesh elements, non-moving
start_mesh = np.array([
    [[1,0], [1,1]],
    [[1,1], [1,2]]
])
end_mesh = np.array([
    [[1,0], [1,1]],
    [[1,1], [1,2]]
])
startpt = np.array([0,0.5])
endpt = np.array([2,0.5])

# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, start_mesh, end_mesh, 10, 10)

In [ ]:
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt)